<a href="https://colab.research.google.com/github/FrancoDiAahumada/etl-Project-GCP/blob/main/etl_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

# **Spark session y Librerias**

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from os import truncate
from pyspark.sql.functions import lit




Spsession = SparkSession.builder.appName('ETL').getOrCreate()

# **Cargar el Spark Context**

In [6]:
SpContext = Spsession.sparkContext

# **Cargar Datos**

In [8]:
data = Spsession.read.csv('cars.csv', header=True, sep=';', inferSchema=True)
data.show(10)

+--------------------+------+---------+------------+----------+------+------------+-----+------+
|                 Car|   MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+------+---------+------------+----------+------+------------+-----+------+
|              STRING|DOUBLE|      INT|      DOUBLE|    DOUBLE|DOUBLE|      DOUBLE|  INT|   CAT|
|Chevrolet Chevell...|  18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|  15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|  18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|  16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|  17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
|    Ford Galaxie 500|  15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|
|    Chevrolet Impala|  14.0| 

# **Informacion de los Datos**

In [9]:
data.printSchema()

root
 |-- Car: string (nullable = true)
 |-- MPG: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Displacement: string (nullable = true)
 |-- Horsepower: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Acceleration: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Origin: string (nullable = true)



In [11]:
data.describe().show()

+-------+--------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------+
|summary|                 Car|               MPG|        Cylinders|      Displacement|       Horsepower|            Weight|      Acceleration|             Model|Origin|
+-------+--------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------+
|  count|                 407|               407|              407|               407|              407|               407|               407|               407|   407|
|   mean|                NULL|23.051231527093602|5.475369458128079| 194.7795566502463|103.5295566502463|2979.4137931034484|15.519704433497521| 75.92118226600985|  NULL|
| stddev|                NULL|   8.4017773522706|1.712159631548529|104.92245837948867|40.52065912106347| 847.0043282393513|2.8033588163425462|3.74873734545

In [23]:

data.select(data.Car,data.Model,data.Cylinders).distinct().show(truncate=False)

+----------------------------+-----+---------+
|Car                         |Model|Cylinders|
+----------------------------+-----+---------+
|Chevrolet Monte Carlo Landau|77   |8        |
|Mazda RX-4                  |77   |3        |
|Datsun 200SX                |81   |4        |
|Hi 1200D                    |70   |8        |
|Ford Mustang II             |75   |8        |
|Pontiac Catalina            |70   |8        |
|Mazda GLC Deluxe            |79   |4        |
|AMC Rebel SST (sw)          |70   |8        |
|Dodge Colt                  |76   |4        |
|Renault Lecar Deluxe        |80   |4        |
|Fiat 128                    |74   |4        |
|Oldsmobile Omega Brougham   |79   |6        |
|Plymouth Satellite (sw)     |70   |8        |
|Volkswagen Super Beetle 117 |71   |4        |
|Chrysler Newport Royal      |72   |8        |
|Volkswagen Rabbit           |76   |4        |
|Honda Accord                |80   |4        |
|Toyota Tercel               |81   |4        |
|Dodge Colt  

# **Agregar Nuevas Columnas**

In [24]:
from pyspark.sql.functions import lit

df = data.withColumn('Continents', lit('America')) \
          .withColumn('currency', lit('Dolar'))

df.show(10, truncate=False)

+-------------------------+------+---------+------------+----------+------+------------+-----+------+----------+--------+
|Car                      |MPG   |Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Continents|currency|
+-------------------------+------+---------+------------+----------+------+------------+-----+------+----------+--------+
|STRING                   |DOUBLE|INT      |DOUBLE      |DOUBLE    |DOUBLE|DOUBLE      |INT  |CAT   |America   |Dolar   |
|Chevrolet Chevelle Malibu|18.0  |8        |307.0       |130.0     |3504. |12.0        |70   |US    |America   |Dolar   |
|Buick Skylark 320        |15.0  |8        |350.0       |165.0     |3693. |11.5        |70   |US    |America   |Dolar   |
|Plymouth Satellite       |18.0  |8        |318.0       |150.0     |3436. |11.0        |70   |US    |America   |Dolar   |
|AMC Rebel SST            |16.0  |8        |304.0       |150.0     |3433. |12.0        |70   |US    |America   |Dolar   |
|Ford Torino            

# **Agrupar Columnas**

In [29]:
df.groupBy( 'Origin','car').count().show(10)

+------+-------------------+-----+
|Origin|                car|count|
+------+-------------------+-----+
|    US|         Ford Pinto|    6|
| Japan|     Datsun B210 GX|    1|
|    US|    Dodge Magnum XE|    1|
|    US|        Ford Torino|    1|
|Europe|          Opel 1900|    2|
|    US|    Mercury Marquis|    1|
| Japan|             Subaru|    2|
|    US|          Ford F108|    1|
|Europe|Volkswagen Rabbit l|    1|
|    US|           Ford LTD|    2|
+------+-------------------+-----+
only showing top 10 rows



# **Eliminar columna**

In [31]:
df = df.drop('Continents')
df.show(10, truncate=False)

+-------------------------+------+---------+------------+----------+------+------------+-----+------+--------+
|Car                      |MPG   |Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|currency|
+-------------------------+------+---------+------------+----------+------+------------+-----+------+--------+
|STRING                   |DOUBLE|INT      |DOUBLE      |DOUBLE    |DOUBLE|DOUBLE      |INT  |CAT   |Dolar   |
|Chevrolet Chevelle Malibu|18.0  |8        |307.0       |130.0     |3504. |12.0        |70   |US    |Dolar   |
|Buick Skylark 320        |15.0  |8        |350.0       |165.0     |3693. |11.5        |70   |US    |Dolar   |
|Plymouth Satellite       |18.0  |8        |318.0       |150.0     |3436. |11.0        |70   |US    |Dolar   |
|AMC Rebel SST            |16.0  |8        |304.0       |150.0     |3433. |12.0        |70   |US    |Dolar   |
|Ford Torino              |17.0  |8        |302.0       |140.0     |3449. |10.5        |70   |US    |Dolar   |
|

# **Ordenar Filas y Count de Datos**

In [33]:
df.orderBy('Horsepower', ascending=False).show(truncate=False)

+---------------------------+------+---------+------------+----------+------+------------+-----+------+--------+
|Car                        |MPG   |Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|currency|
+---------------------------+------+---------+------------+----------+------+------------+-----+------+--------+
|STRING                     |DOUBLE|INT      |DOUBLE      |DOUBLE    |DOUBLE|DOUBLE      |INT  |CAT   |Dolar   |
|Volvo 244DL                |22.0  |4        |121.0       |98.00     |2945. |14.5        |75   |Europe|Dolar   |
|Ford Grenada               |18.5  |6        |250.0       |98.00     |3525. |19.0        |77   |US    |Dolar   |
|Toyota Corolla Mark II (sw)|23.0  |4        |120.0       |97.00     |2506. |14.5        |72   |Japan |Dolar   |
|AMC Hornet                 |18.0  |6        |199.0       |97.00     |2774. |15.5        |70   |US    |Dolar   |
|Mazda RX2 Coupe            |19.0  |3        |70.00       |97.00     |2330. |13.5        |72   |

In [37]:
df.groupBy('origin').count().orderBy('count',ascending = False).show()

+------+-----+
|origin|count|
+------+-----+
|    US|  254|
| Japan|   79|
|Europe|   73|
|   CAT|    1|
+------+-----+



# **Filtrado de Datos**

In [60]:

filtered_df = df.filter((df.Origin == 'US')&(df.Cylinders == 4))
filtered_df.show(10, truncate=False)


print("\nFilas con datos nulos en Horsepower:")
df.filter(df.Horsepower.isNull()).show(truncate=False)


row_count = df.count()
print(f"\nNumero de filas: {row_count}")


col_count = len(df.columns)
print(f"Numero de columnas: {col_count}")

print("\nAnalisis descriptivo de los datos:")
df.describe().show()

+-------------------+----+---------+------------+----------+------+------------+-----+------+--------+
|Car                |MPG |Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|currency|
+-------------------+----+---------+------------+----------+------+------------+-----+------+--------+
|Chevrolet Vega 2300|28.0|4        |140.0       |90.00     |2264. |15.5        |71   |US    |Dolar   |
|Ford Pinto         |25.0|4        |98.00       |0         |2046. |19.0        |71   |US    |Dolar   |
|Chevrolet Vega (sw)|22.0|4        |140.0       |72.00     |2408. |19.0        |71   |US    |Dolar   |
|Mercury Capri 2000 |23.0|4        |122.0       |86.00     |2220. |14.0        |71   |US    |Dolar   |
|Plymouth Cricket   |26.0|4        |91.00       |70.00     |1955. |20.5        |71   |US    |Dolar   |
|Dodge Colt Hardtop |25.0|4        |97.50       |80.00     |2126. |17.0        |72   |US    |Dolar   |
|Chevrolet Vega     |20.0|4        |140.0       |90.00     |2408. |19.5  

In [62]:
# Guardar en parquet
Salida_parquet = 'transformed_cars.parquet'
df.write.mode('overwrite').parquet(Salida_parquet)

print(f"Df guardado en Parquet{Salida_parquet}")


Df guardado en Parquettransformed_cars.parquet
